In [20]:
import pandas as pd
import os
from tqdm.notebook import tqdm
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import texttable
import latextable

tags = {
    "loss": "Loss_MB/train_phase/train_stream/Task000",
    "final_accuracy": "Accuracy_On_Trained_Experiences/eval_phase/test_stream/Task000",
}

def load_event_file(experiment_directory: str) -> EventAccumulator:
    """
    Loads the event file from a given experiment directory.
    """
    ea: EventAccumulator = EventAccumulator(experiment_directory)
    ea.Reload()
    return ea

def final_metrics(ea: EventAccumulator) -> dict:
    """
    Returns the final metrics from the event file.
    """
    result = {}
    for tag_name, tag in tags.items():
        if tag in ea.Tags()['scalars']:
            result[tag_name] = ea.Scalars(tag)[-1].value
    return result



In [2]:
experiment_logs = "experiment_logs"

records = []

# Loop over each directory in a directory
for experiment in tqdm(os.listdir(os.path.join(experiment_logs))):
    experiment_code = experiment.split("_")
    i, prune_strategy, dataset, arch, strategy = experiment_code


    record = {
        "dataset": dataset,
        "architecture": arch,
        "strategy": strategy,
        "prune_strategy": prune_strategy,
        "id": i,
    }

    try:
        ea = load_event_file(os.path.join(experiment_logs, experiment))
        record.update(final_metrics(ea))
    except KeyError as e:
        print(f"Could not load required metric {e} in '{experiment}'")

    records.append(record)

df = pd.DataFrame.from_dict(records)
df

  0%|          | 0/130 [00:00<?, ?it/s]

,dataset,architecture,strategy,prune_strategy,id,loss,final_accuracy
0,splitEmbeddedCIFAR100,AE,cumulative,EXP,0057,1.267279,0.326700
1,splitEmbeddedCIFAR100,AE,finetuning,EXP,0068,0.680026,0.142400
2,splitEmbeddedCIFAR100,AE,cumulative,EXP,0106,NaN,NaN
3,splitCORe50,VAE,finetuning,EXP,0030,0.239605,0.075069
4,splitEmbeddedCIFAR100,AE,taskInference,EXP,0071,0.835625,0.161500
...,...,...,...,...,...,...,...
125,splitCIFAR10,AE,taskInference,EXP,0040,0.342900,0.712300
126,splitEmbeddedCIFAR100,AE,cumulative,EXP,0049,1.520590,0.323200
127,splitEmbeddedCIFAR100,AE,taskInference,EXP,0070,0.642157,0.706000
128,splitCIFAR100,VAE,taskOracle,EXP,0037,1.664222,0.591000


In [3]:
df.dropna().sort_values(by='id').groupby(['dataset', 'architecture', 'strategy']).max().pivot_table(index=['architecture', 'strategy'], columns=['dataset'], values='final_accuracy').to_csv('accuracy.csv')

In [4]:
df.to_csv("results/results.csv")

In [32]:
# 
# Regular Experiment
# 
for strategy in ["cumulative", "taskOracle", "finetuning", "taskInference"]:
    for arch in ["AE", "VAE"]:
        for dataset in [
            "splitFMNIST",
            "splitCIFAR10",
            "splitCIFAR100",
            "splitCORe50",
            "splitEmbeddedCIFAR100",
            "splitEmbeddedCORe50"]:
            try:
                result = df[df['dataset'] == dataset]
                result = result[result['prune_strategy'] == "EXP"]
                result = result[result['strategy'] == strategy]
                result = result[result['architecture'] == arch]
                accuracy = result.sort_values(by='id').iloc[-1]["final_accuracy"]
                # accuracy = result.sort_values("id")[-1]["accuracy"]
                # display(result.sort_values(by='id'))
                print(f"{accuracy:0.4f}", end="\t")
            except:
                continue
        print()

0.9151	0.8721	0.6167	0.3560	0.4210	0.7720	
0.8989	0.8725	0.6106	0.3604	0.3974	0.7593	
0.9938	0.9375	0.5475	0.4322	0.5513	0.6042	
0.9936	0.9382	0.5910	0.4032	0.4735	0.5191	
0.1995	0.1939	0.0899	0.0745	0.0841	0.0881	
0.1995	0.1945	0.0891	0.0751	0.0849	0.0578	
0.9597	0.7123	0.2351	0.0553	0.4392	0.4191	
0.9927	0.7295	0.3091	0.0531	0.3626	0.3756	


In [39]:
# 
# Equal Prune Experiment
# 
def table_equal_prune(experiment_results: pd.DataFrame):
    df = experiment_results.copy()

    equal_prune_table = texttable.Texttable()
    columns = ["splitFMNIST", "splitCIFAR10", "splitCIFAR100", "splitCORe50", "splitEmbeddedCIFAR100", "splitEmbeddedCORe50"]
    
    equal_prune_table.header(["", ""] + columns)

    for strategy in ["taskOracle", "taskInference"]:
        for arch in ["AE", "VAE"]:
            row = [strategy, arch]
            for dataset in columns:
                try:
                    result = df.loc[
                        (df['dataset'] == dataset) &
                        (df['prune_strategy'] == "EP") &
                        (df['strategy'] == strategy) &
                        (df['architecture'] == arch)
                    ]
                    accuracy = result.sort_values(by='id').iloc[-1]["final_accuracy"]
                    row.append(f"{accuracy*100:0.1f}\%")
                except:
                    row.append(f"NA")
            equal_prune_table.add_row(row)

    print(equal_prune_table.draw())

    print(latextable.draw_latex(
        equal_prune_table,
        caption="Equal Prune Experiment",
        label="tab:equal_prune_experiment",
        use_booktabs=True
    ))

table_equal_prune(df[df["prune_strategy"]=="EP"])

+----------+-----+----------+----------+---------+---------+---------+---------+
|          |     | splitFMN | splitCIF | splitCI | splitCO | splitEm | splitEm |
|          |     |   IST    |   AR10   | FAR100  |  Re50   | beddedC | beddedC |
|          |     |          |          |         |         | IFAR100 |  ORe50  |
+==========+=====+==========+==========+=========+=========+=========+=========+
| taskOrac | AE  | 99.3\%   | 93.4\%   | 55.7\%  | 50.7\%  | NA      | NA      |
| le       |     |          |          |         |         |         |         |
+----------+-----+----------+----------+---------+---------+---------+---------+
| taskOrac | VAE | 99.3\%   | 91.7\%   | 53.0\%  | 54.1\%  | NA      | NA      |
| le       |     |          |          |         |         |         |         |
+----------+-----+----------+----------+---------+---------+---------+---------+
| taskInfe | AE  | 99.3\%   | 43.0\%   | 25.7\%  | 8.2\%   | NA      | NA      |
| rence    |     |          

In [31]:
df[df['prune_strategy'] == "EP"]


,dataset,architecture,strategy,prune_strategy,id,loss,final_accuracy
21,splitFMNIST,VAE,taskOracle,EP,0138,NaN,0.993500
32,splitCIFAR100,VAE,taskInference,EP,0148,NaN,0.427600
35,splitFMNIST,VAE,taskInference,EP,0146,NaN,0.993000
36,splitFMNIST,AE,taskInference,EP,0142,NaN,0.993100
45,splitCIFAR100,VAE,taskOracle,EP,0140,NaN,0.530200
54,splitCORe50,AE,taskInference,EP,0145,NaN,0.082474
57,splitFMNIST,AE,taskOracle,EP,0134,NaN,0.993100
75,splitCIFAR100,AE,taskInference,EP,0144,NaN,0.256600
86,splitCIFAR10,AE,taskOracle,EP,0135,NaN,0.933700
88,splitCORe50,VAE,taskOracle,EP,0141,NaN,0.540648
